## Project Goal

In this notebook, we will read in json data from OpenCourseAPI.
Then we will design a relational model for the data and store in a sqlite database

    - remember to run `jupyter nbconvert --to script model_open_course_data.ipynb` to output the notebook into a python script

In [133]:
import json
import sqlite3
from sqlite3 import Error

## Get the data
- from https://opencourse.dev/fh/classes

In [134]:
from pathlib import Path     

data_path = Path().absolute() / "fh-classes.json" # json data from https://opencourse.dev/fh/classes

with data_path.open() as f:
    data = json.load(f)

### Lets take a look at the structure of the json data: 

In [135]:
data[230]

{'CRN': 21845,
 'raw_course': 'BIOL F040B04V',
 'dept': 'BIOL',
 'course': '40B',
 'section': '04V',
 'title': 'Human Anatomy & Physiology II',
 'units': 5.0,
 'start': '09/21/2020',
 'end': '12/11/2020',
 'seats': 0,
 'wait_seats': 1,
 'status': 'waitlist',
 'times': [{'days': 'MW',
   'start_time': '12:00 PM',
   'end_time': '01:50 PM',
   'instructor': ['Connie La'],
   'location': 'FH ONLINE'},
  {'days': 'W',
   'start_time': '09:00 AM',
   'end_time': '11:50 AM',
   'instructor': ['Connie La'],
   'location': 'FH ONLINE'}]}

### Let's extract relevant data
- CRN
- raw_course:  full course name
- dept: Abbreviated department
- course: course number
- section: section number and modality (online/virtual)
- title
- units
- start: start date
- end: end date
- seats: seats available
- wait_seats: waitlist seat available
- status: 'waitlist' or 'open'
- times: a list of dicts with keys ('days', 'start_time', 'end_time', 'instructor', 'location')

## create a connection to a sqlite database

In [136]:
def sql_connect(db_path):
    try:
        conn = sqlite3.connect(db_path)
        print("Connected to " + db_path)
        return conn
    except Error as e:
        print(e)

In [137]:
db_path = "/Users/dnghiem/Projects/schedule_analysis/fh-classes.db"

conn = sql_connect(db_path)
cur = conn.cursor()

Connected to /Users/dnghiem/Projects/schedule_analysis/fh-classes.db


In [138]:
# Enable foreign key constraint
# PRAGMA foreign_keys is 1 if enabled

cur.execute("PRAGMA foreign_keys = ON")
cur.execute("PRAGMA foreign_keys")
result = cur.fetchall()
print(result)

[(1,)]


## Let's Create the tables

We will normalize the data by creating several tables:
- class : each entry identified by a CRN code
- meetings: the day, time, and locationfor each class meeting
- instructor class: the instructor name and a class they are teaching

In [139]:
table_class ="""
    CREATE TABLE IF NOT EXISTS classes (
        crn integer PRIMARY KEY,
        dept text,
        course text,
        section text,
        title text,
        units real,
        start text,
        end text,
        seats integer,
        wait_seats integer,
        status text
    );"""

cur.execute("DROP TABLE IF EXISTS classes")
cur.execute(table_class)

In [140]:
# Lets inspect the class meetings from data

meetings = []

for cls in data:
    course_id = cls.get('CRN')
    for time in cls.get('times', None):
        time['CRN'] = course_id
        meetings.append(time)
            
meetings[155]

{'days': 'TBA',
 'start_time': 'TBA',
 'end_time': 'TBA',
 'instructor': ['Robbie Reid'],
 'location': 'FH ONLINE',
 'CRN': 20161}

In [141]:
# Since many classes may have "TBA" listed as their days and start times, 
# instead of using a composite key we will create an id for each meeting 

create_meetings = """
    CREATE TABLE IF NOT EXISTS meetings (
        id INTEGER PRIMARY KEY,
        crn integer NOT NULL,
        days text,
        start_time text,
        end_time text,
        location text,
        FOREIGN KEY (crn)
            REFERENCES classes (crn)
        )"""

cur.execute("DROP TABLE IF EXISTS meetings")
cur.execute(create_meetings)

In [142]:
create_instructor_class = """
    CREATE TABLE IF NOT EXISTS instructor_class (
        name text,
        crn integer,
        PRIMARY KEY (name, crn),
        FOREIGN KEY (crn)
            REFERENCES classes (crn)
        )"""

cur.execute("DROP TABLE IF EXISTS instructor_class")
cur.execute(create_instructor_class)

In [143]:
# Check that the table is there

cur.execute("SELECT name FROM sqlite_master WHERE type='table'")

print(cur.fetchall())

[('classes',), ('meetings',), ('instructor_class',)]


## Let's insert into the class table

In [144]:
cls = data[89]
cls 

{'CRN': 22033,
 'raw_course': 'ALCB F431Y04V',
 'dept': 'ALCB',
 'course': '431Y',
 'section': '04V',
 'title': 'Analysis of Current Events',
 'units': 0.0,
 'start': '09/21/2020',
 'end': '12/11/2020',
 'seats': 11,
 'wait_seats': 0,
 'status': 'open',
 'times': [{'days': 'Th',
   'start_time': '01:00 PM',
   'end_time': '02:50 PM',
   'instructor': ['Diane C Smith'],
   'location': 'FO ONLINE',
   'CRN': 22033}]}

In [145]:
# Insert into the classes table using named parameters from data

insert_classes ="""
    INSERT INTO classes VALUES (
        :CRN,
        :dept,
        :course,
        :section,
        :title,
        :units,
        :start,
        :end,
        :seats,
        :wait_seats,
        :status
        )"""

for cls in data:
    cur.execute(insert_classes, cls)

In [146]:
cls = data[89]
cls

{'CRN': 22033,
 'raw_course': 'ALCB F431Y04V',
 'dept': 'ALCB',
 'course': '431Y',
 'section': '04V',
 'title': 'Analysis of Current Events',
 'units': 0.0,
 'start': '09/21/2020',
 'end': '12/11/2020',
 'seats': 11,
 'wait_seats': 0,
 'status': 'open',
 'times': [{'days': 'Th',
   'start_time': '01:00 PM',
   'end_time': '02:50 PM',
   'instructor': ['Diane C Smith'],
   'location': 'FO ONLINE',
   'CRN': 22033}]}

In [147]:
# Debug: Which classes have the same instructor appear multiple times in the meetings of the class?
multiples = dict()

for cls in data:
    crn = cls['CRN']
    
    if len(cls['times']) > 1:
        multiples[crn] = cls['times']

# Debug: Which classes have a suffix that indicates fully online or hybrid?

classes = dict()

for cls in data:
    raw = cls['raw_course']
    suffix = raw[-1]
    
    times = cls['times']
    
    has_tba = False
    
    for time in times:
        if time['start_time'] == 'TBA':
            has_tba = True
    if has_tba:
        classes[raw] = times
        
suffixes = set()
for cls in classes.keys():
    suffixes.add(cls[-1])

print(suffixes)


# Which classes have 'TBA' for all of their start times?
# Nearly all the classes that have all 'TBA' start times are classes with 'W' at the end of the section name
# 
all_tba = [] # list of classes with all 'TBA' start times

for cls in data:
    raw = cls['raw_course']
    
    has_tba = [time['start_time']=='TBA' for time in cls['times']]
    if all(has_tba):
       all_tba.append(raw) 

all_tba_no_W = [name for name in all_tba if name[-1] != 'W']

all_tba_no_W

        

{'W', '1', 'T', '3', '4', 'Z', 'Y', 'D', '0', '2'}


['ACTG F070R01',
 'ACTG F071R01',
 'ACTG F072R01',
 'ACTG F073R01',
 'ANTH F070R01',
 'ANTH F071R01',
 'ANTH F072R01',
 'ANTH F073R01',
 'ART F070R01',
 'ART F070R02',
 'ASTR F070R01',
 'ASTR F071R01',
 'ASTR F072R01',
 'ASTR F073R01',
 'ATHL F070R03',
 'ATHL F072R01',
 'BUSI F070R01',
 'BUSI F071R01',
 'BUSI F072R01',
 'BUSI F073R01',
 'D H F071R01',
 'ECON F070R01',
 'ECON F071R01',
 'ECON F072R01',
 'ECON F073R01',
 'EMS F064A01',
 'ENGR F070R01',
 'ENGR F071R01',
 'ENGR F072R01',
 'ENGR F073R01',
 'ENGL F001A61D',
 'ENGL F001A62D',
 'ENGL F073R01',
 'GEOG F070R01',
 'GEOG F071R01',
 'GEOG F072R01',
 'GEOG F073R01',
 'ITRN F050.01',
 'ITRN F050.04',
 'ITRN F051.01',
 'ITRN F051.03',
 'ITRN F052.01',
 'ITRN F052.02',
 'ITRN F053.02',
 'ITRN F053.03',
 'ITRN F054.02',
 'MATH F070R01',
 'MATH F071R01',
 'MATH F072R01',
 'MATH F073R01',
 'MUS F070R01',
 'NCBS F405.50',
 'NCCS F406.50',
 'NCLA F406A14',
 'PHIL F070R01',
 'PHIL F071R01',
 'PHIL F072R01',
 'PHIL F073R01',
 'PHYS F070R01',


In [148]:
# Insert into the instructor_class table

for cls in data:
    crn = cls['CRN']
    
    instructors = set()
    
    # execute one insert statement for each instructor of the class
    for time in cls['times']:
        for name in time['instructor']:
            instructors.add(name)
    for name in instructors:
        cur.execute("INSERT INTO instructor_class VALUES (?, ?)", (name, crn))
        
cur.execute("SELECT * FROM instructor_class")
result = cur.fetchall()
result

[('Joe L Mayer', 20238),
 ('Lisa Ann Drake', 20001),
 ('Paul Henry Ponleithner', 20574),
 ('Sara Seyedin', 20002),
 ('Sara Seyedin', 20003),
 ('Sara Seyedin', 20281),
 ('Sara Seyedin', 20341),
 ('Sara Seyedin', 21163),
 ('Lisa Ann Drake', 20441),
 ('Lisa Ann Drake', 20224),
 ('Lisa Ann Drake', 20475),
 ('Lisa Ann Drake', 20608),
 ('Lloyd Zachary Benson', 20005),
 ('Lloyd Zachary Benson', 20247),
 ('Arthur Joseph Ardizzone', 20642),
 ('Huipin Emily Polivka', 20643),
 ('Jeffrey Scott Bullock', 20309),
 ('Farima Fakoor', 20310),
 ('Jeffrey Scott Bullock', 20618),
 ('Arthur Joseph Ardizzone', 21701),
 ('Sean Paul Bowman', 20202),
 ('Sean Paul Bowman', 20442),
 ('Joe L Mayer', 20249),
 ('Paul Henry Ponleithner', 20456),
 ('Nancy Narges Ghodrat', 20006),
 ('Nancy Narges Ghodrat', 20307),
 ('Subha Gopal', 20443),
 ('Subha Gopal', 20444),
 ('Juston Elliot Glass', 20445),
 ('Juston Elliot Glass', 20446),
 ('Gabriel Negrete', 20007),
 ('Gabriel Negrete', 20248),
 ('Farima Fakoor', 20008),
 ('Olg

## Insert into Meetings

In [149]:
# Insert into the meetings table

# TODO:  Many of the class meetings 'TBA' in the start and end times because they are fully online classes.
#        The scraper needs to return a value that indicates a class in online and has no in person meetings

for cls in data:
    crn = cls['CRN']
     
    for time in cls['times']:
        
        days = time.get('days').replace('Th', 'H')
        
        cur.execute("INSERT INTO meetings VALUES (null, ?, ?, ?, ?, ?)", (crn, days, time['start_time'], time['end_time'], time['location']))
     
cur.execute("select * from meetings")
result = cur.fetchall()
result

[(1, 20238, 'TBA', 'TBA', 'TBA', 'FC ONLINE'),
 (2, 20001, 'TBA', 'TBA', 'TBA', 'FC ONLINE'),
 (3, 20574, 'TBA', 'TBA', 'TBA', 'FC ONLINE'),
 (4, 20002, 'TBA', 'TBA', 'TBA', 'FC ONLINE'),
 (5, 20003, 'TBA', 'TBA', 'TBA', 'FC ONLINE'),
 (6, 20281, 'TBA', 'TBA', 'TBA', 'FC ONLINE'),
 (7, 20341, 'TBA', 'TBA', 'TBA', 'FC ONLINE'),
 (8, 21163, 'TBA', 'TBA', 'TBA', 'FC ONLINE'),
 (9, 20441, 'TBA', 'TBA', 'TBA', 'FC ONLINE'),
 (10, 20224, 'TBA', 'TBA', 'TBA', 'FC ONLINE'),
 (11, 20475, 'TBA', 'TBA', 'TBA', 'FC ONLINE'),
 (12, 20608, 'TBA', 'TBA', 'TBA', 'FC ONLINE'),
 (13, 20005, 'TBA', 'TBA', 'TBA', 'FC ONLINE'),
 (14, 20247, 'TBA', 'TBA', 'TBA', 'FC ONLINE'),
 (15, 20642, 'TBA', 'TBA', 'TBA', 'FC ONLINE'),
 (16, 20643, 'TBA', 'TBA', 'TBA', 'FC ONLINE'),
 (17, 20309, 'TBA', 'TBA', 'TBA', 'FC ONLINE'),
 (18, 20310, 'TBA', 'TBA', 'TBA', 'FC ONLINE'),
 (19, 20618, 'TBA', 'TBA', 'TBA', 'FC ONLINE'),
 (20, 21701, 'TBA', 'TBA', 'TBA', 'FC ONLINE'),
 (21, 20202, 'TBA', 'TBA', 'TBA', 'FC ONLINE'),
 

In [150]:
query ="""
    SELECT * FROM meetings
    INNER JOIN (
        SELECT *
        FROM instructor_class
        INNER JOIN classes
        ON instructor_class.crn = classes.crn
        WHERE instructor_class.name = 'Daniel Nghiem'
    ) AS B
    ON meetings.crn = B.crn
    """


cur.execute(query)
result = cur.fetchall()
print(len(result))
print(result)

1
[(933, 20769, 'TH', '01:30 PM', '03:45 PM', 'FH ONLINE', 'Daniel Nghiem', 20769, 20769, 'MATH', '1B', '03V', 'Calculus', 5.0, '09/21/2020', '12/11/2020', 0, 8, 'waitlist')]


# TO DO
- Ideas?


## Analysis in pandas

- we can use `pandas.read_sql` to read sql-style queries

In [151]:
import pandas as pd

df_classes = pd.read_sql("SELECT * FROM classes", conn)

df_classes.head()

,crn,dept,course,section,title,units,start,end,seats,wait_seats,status
0,20001,ACTG,1A,02W,Financial Accounting I,5.0,09/21/2020,12/11/2020,0,15,waitlist
1,20002,ACTG,1A,04W,Financial Accounting I,5.0,09/21/2020,12/11/2020,8,20,open
2,20003,ACTG,1A,05W,Financial Accounting I,5.0,09/21/2020,12/11/2020,8,20,open
3,20005,ACTG,1C,02W,Managerial Accounting,5.0,09/21/2020,12/11/2020,0,14,waitlist
4,20006,ACTG,60,01W,Accounting for Small Business,5.0,09/21/2020,12/11/2020,1,10,open


In [153]:
df_meetings = pd.read_sql("SELECT * FROM meetings", conn)

df_meetings.head()

,id,crn,days,start_time,end_time,location
0,1,20238,TBA,TBA,TBA,FC ONLINE
1,2,20001,TBA,TBA,TBA,FC ONLINE
2,3,20574,TBA,TBA,TBA,FC ONLINE
3,4,20002,TBA,TBA,TBA,FC ONLINE
4,5,20003,TBA,TBA,TBA,FC ONLINE


In [125]:
conn.close()